In [ ]:
'''
    Cloud function para cargar data de OlivIA en panel

    Autores: Diego Alvarez, Eduardo Estay, Cristobal Herreros
'''
# Imports
import os
from dotenv import load_dotenv
from utility import valid_event, instantiate_api_clients, clean_data
from storage import load_data_from_storage
from bigquery import send_data_to_bq

# Variables de entorno
load_dotenv()

# Nombre del archivo de credenciales (debe estar en mismo directorio que cloud function)
SERVICE_ACCOUNT_CREDENTIALS_FILE = os.getenv('SERVICE_ACCOUNT_CREDENTIALS_FILE')

# Nombre del bucket
TARGET_BUCKET_NAME=os.getenv('TARGET_BUCKET_NAME')

# Nombre de la tabla en bigQuery
TABLE_NAME = os.getenv('TABLE_NAME')

# Punto de entrada
def main(event, context):
    # Validar evento al iniciar la cloud function
    valid_event(event, context)

    # Instanciar clientes de API
    storage_client, bigquery_client = instantiate_api_clients()

    # Tomar data del bucket y cargarla en memoria
    filename = event["name"]
    dataframe, load_success = load_data_from_storage(storage_client, TARGET_BUCKET_NAME, filename)

    if load_success:
        # Limpiar la data
        clean_dataframe, clean_success = clean_data(dataframe)

        # Si no se logró limpiar la data, detener ejecución
        if not clean_success:
            return

        # Cargar data en bigQuery
        send_data_to_bq(bigquery_client, TABLE_NAME, clean_dataframe)